<a href="https://colab.research.google.com/github/margotfournier/Kaggle_Essai/blob/master/DameSermonde_loras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from diffusers import FluxPipeline
import torch
from huggingface_hub import notebook_login
import matplotlib.pyplot as plt
from PIL import Image
import random
import os


In [ ]:
# 🔑 Authentification Hugging Face (pour accès privé)
notebook_login()  # ton token Hugging Face

In [ ]:
# 🔧 Charge le modèle base (ici compatible Flux — adapte si besoin)
pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",  # ou un autre modèle compatible avec ton LoRA
    torch_dtype=torch.float16,
).to("cuda")

# 📥 Charge le LoRA depuis ton dépôt Hugging Face (privé !)
pipe.load_lora_weights("margotfournier/DameSermonde-v0",
                       weight_name="DameSermonde_24gb_000001750.safetensors",
                       adapter_name="DameSermonde"
                      )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Display Dame Sermonde

In [ ]:
# 🎨 Utilisation : active le LoRA dans le prompt
prompt = "portrait of DameSermonde, medieval dress, ethereal lighting"
image = pipe(prompt).images[0]
display(image)

# Generate instagram images

In [ ]:
# @title String fields

prompt = 'Dame Sermonde standing alone in a candle-lit chapel, wearing an ornate wedding dress made of moon-white silk, embroidered with faded roses and silver thread. Her expression is solemn, almost defiant, as she stands at the altar, bouquet loosely held in one hand. Her hair is intricately braided and wrapped in a delicate pearl-embroidered headdress, with a sheer veil cascading behind her. The chapel is dim and cold, with stained glass windows casting colored light on the stone floor. Candles flicker along the walls. The atmosphere is melancholic and dramatic, with a sense of silent resistance. High gothic arches, detailed textures, medieval realism, soft golden light, cinematic.' # @param {type:"string"}


In [ ]:
prompt = """Dame Sermonde standing alone in a candle-lit chapel, wearing an ornate wedding dress made of moon-white silk, embroidered with faded roses and silver thread. Her expression is solemn, almost defiant, as she stands at the altar, bouquet loosely held in one hand. Her hair is intricately braided and wrapped in a delicate pearl-embroidered headdress, with a sheer veil cascading behind her. The chapel is dim and cold, with stained glass windows casting colored light on the stone floor. Candles flicker along the walls. The atmosphere is melancholic and dramatic, with a sense of silent resistance. High gothic arches, detailed textures, medieval realism, soft golden light, cinematic.
"""

In [ ]:
# 🖼️ Generate + save
weight = 0.7
base_seed = 42

# 📁 Create output folder
output_dir = "damesermonde_outputs/troispics_green"
os.makedirs(output_dir, exist_ok=True)

# 📝 Enregistrer le prompt une seule fois dans output_dir/prompt.txt
with open(os.path.join(output_dir, "prompt.txt"), "w", encoding="utf-8") as f:
    f.write(prompt)

images = []
for i in range(8):
    seed = base_seed + i
    pipe.set_adapters(["DameSermonde"], adapter_weights=[weight])
    generator = torch.manual_seed(seed)

    img = pipe(prompt, generator=generator).images[0]

    filename = f"damesermonde_w{int(weight*100)}_s{seed}.png"
    save_path = os.path.join(output_dir, filename)
    img.save(save_path)

    images.append((img, filename))



In [ ]:
# 🖼️ Affichage dans une grille 4x2
fig, axes = plt.subplots(2, 4, figsize=(18, 9))
for ax, (img, filename) in zip(axes.flatten(), images):
    ax.imshow(img)
    ax.set_title(filename, fontsize=9)
    ax.axis("off")

plt.tight_layout()
plt.show()